Устанавливаем Lark

In [ ]:
pip install lark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
from lark import Lark
import xml.etree.ElementTree as ET
import lark


Ниже парсер Ларк, который работает принимая 1 строчку и выдает дерево по словарному входу.
parser - это грамматика, которую я прописал.

In [41]:


parser = Lark(r"""
        start: entry
        entry: form gramgrp? sense?
        form: orth 
        orth: HEADWORD ","*

        gramgrp: ("[" gen ";" number "],") tns ";"
                    | ("[" number "],") tns ";" gen ";"
                    | case ";" number ";"
        case: CASE_NOUN
        gen: GRAM
        number: GRAM
        tns: GRAM

        sense: lbl? (def extent)* cit*
        lbl: LABEL
        def: WORD+ [";"]+ ";"
        extent: EXT+ ";"*
        LABEL: "несов." | "сов."
        cit: "♦" EXT+

        HEADWORD: /[0-9\/А-Я-IΙ]+/
        GRAM: /[-а-яIΙ~\.\s\/()\,]+/
        WORD: /[0-9\/А-Яа-яё()\s,-\.]+/
        EXT: /[\/А-Яа-яёIΙ()~\s,-;!\.«»]+/
        CASE_NOUN: /[-а-я\.\s\/\,]+/

        // -------- Importing basic data types and ignoring whitespaces --------
        %import common.WS
        %ignore WS
    """, parser="earley")



In [42]:
'''
  Функция обхода дерева lark
'''

def convert_to_xml(lark_tree_):
  if type(lark_tree_) is lark.lexer.Token:
    return lark_tree_.value
  root = ET.Element(lark_tree_.__dict__['data'].value) # Объявляем корневой тег
  for clave in lark_tree_.__dict__['children']: # Обходим ветви дерева
    trs = convert_to_xml(clave) # Рекурсивный вызов функции
    if type(trs) is str: # Если тег конечный, добавляем его как текст
      root.text = trs
    else:
      root.append(trs)

  return root

Теперь загружаем в ФАЙЛЫ (слева) наш текстовый файл со словарными записями, открываем и парсим его, конвертируем строки в формат тегов xml

In [43]:
with open('/content/test2.txt', 'r') as rw:
  file_dict = rw.readlines()

body = ET.Element('body')
for row in file_dict:
  tree = parser.parse(row.strip()) # Парсим строку
  row_xml = convert_to_xml(tree) # Конвертируем строку в  xml
  body.append(row_xml) # Добавляем строку в общий xml
  
text = ET.Element('text')
text.append(body)

In [44]:
ET.indent(text, space = '    ')

In [45]:
ET.dump(text)

<text>
    <body>
        <start>
            <entry>
                <form>
                    <orth>АБДАЛГЯГI</orth>
                </form>
                <gramgrp>
                    <case>-ли, -ла</case>
                    <number>-яни</number>
                </gramgrp>
                <sense>
                    <def>1) психопат, тронутый (прост.), юродивый</def>
                    <extent> 2) простак, простофиля; дураковатый, дурашливый, придурковатый; ~ сайри хIу, ахмах! простофиля ты, дурачина!</extent>
                </sense>
            </entry>
        </start>
        <start>
            <entry>
                <form>
                    <orth>АБЗ/БИКI-ЕС</orth>
                </form>
                <gramgrp>
                    <gen>~икΙ-ес, ~рикΙ-ес</gen>
                    <number> ~дикΙ-ес</number>
                    <tns>-ар, -ули, -уси, -ен(-ена/я)</tns>
                </gramgrp>
                <sense>
                    <lbl>несов.</lbl>
              

In [34]:
teiHeader = '''
<TEI>
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Даргинско-русский словарь</title>
            </titleStmt>
            <publicationStmt>
                <p>Школа лингвистики НИУ ВШЭ</p>
            </publicationStmt>
            <sourceDesc>
                <p>Создан из словаря в формате DOC</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
</TEI>
'''

In [35]:
TEI = ET.fromstringlist(teiHeader)

In [36]:
TEI.append(text)

In [37]:
ET.dump(TEI)

<TEI>
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Даргинско-русский словарь</title>
            </titleStmt>
            <publicationStmt>
                <p>Школа лингвистики НИУ ВШЭ</p>
            </publicationStmt>
            <sourceDesc>
                <p>Создан из словаря в формате DOC</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
<text>
    <body>
        <start>
            <entry>
                <form>
                    <orth>АБДАЛГЯГI</orth>
                </form>
                <gramgrp>
                    <case>-ли, -ла</case>
                    <number>-яни</number>
                </gramgrp>
                <sense>
                    <def>1) психопат, тронутый (прост.), юродивый</def>
                    <extent> 2) простак, простофиля; дураковатый, дурашливый, придурковатый; ~ сайри хIу, ахмах! простофиля ты, дурачина!</extent>
                </sense>
            </entry>
        </start>
        <star

In [38]:
dict_xml_darg = ET.tostring(TEI, encoding="utf-8", method="xml").decode(encoding="utf-8") # Переводим формат (ET.element - сродни формату BeatifoulSoup) в str

In [39]:
# Сохраняем в xml-файл
fp = open('dictDarg.xml', 'w')
fp.write(dict_xml_darg)
fp.close()